The next step is to take a more reasonable midi file, then for each note do the calculations and predict what/when the next note will be. Then compare to the actual next note

In [1]:
import mido
import matplotlib.pyplot as plt
import numpy as np
import time

import smart_loop_test

%matplotlib inline

In [2]:
mf = mido.MidiFile('cycle-test_2020-04-15.mid')

In [3]:
track = mf.tracks[0]

In [30]:
current_time = 0
note_ons = []
notes = []
# The index of each note_on message in track
note_is = []

for i in range(len(track)):
    current_time = current_time + track[i].time
    #print("{}, time: {}".format(track[i], current_time))
    if(track[i].type == "note_on"):
        note_ons.append(current_time)
        notes.append(track[i].note)
        note_is.append(i)

In [5]:
us_per_beat = 500000.0
seconds_per_tick = 1e-6 * us_per_beat / mf.ticks_per_beat

In [6]:
#for i in range(1,len(note_ons)):
#    print(i)
#    shifts = smart_loop_test.get_sequence_self_overlaps(np.array(note_ons[0:i], dtype=float))
#    p = [smart_loop_test.labelled_seq_product(note_ons[:i], notes[:i], s, 50) for s in shifts]   

In [7]:
i = 50
shifts, next_is = smart_loop_test.get_sequence_self_overlaps(np.array(note_ons[0:i], dtype=float))
p = [smart_loop_test.labelled_seq_product(note_ons[:i], notes[:i], s, i) for s in shifts]   

In [8]:
max_prod = np.argmax(p)
period = shifts[max_prod]

Now that we know what the period is, what is the index of the 'next' note in the sequence.

In fact, we should be able to get at least two of these (more from the current sequence). What is the period in indices? It should be max_prod, right?

(At some point we'll have to deal with the fact we have other messages than note_ons).



In [43]:
next_i = next_is[max_prod]
# \forall j, notes[next_is[max_prod] + j] == notes[i + j]

# This is the index of the next shift that produces a repetition
# TODO handle case that it's the last index of shifts (could be a 
# repetition, or could be that the next repetition is way outside 
# of the shifts window)
nextshift_i = smart_loop_test.glbi(shifts, shifts[max_prod] * 2)


In [56]:
# What is the sequence that we would create if we pasted a repetition onto 
splice_track = track[:note_is[next_is[i]]] + track[note_is[next_i]:]

That seems to work ok? Hard to tell for sure

## Play it

In [9]:
port = mido.open_output('Microsoft GS Wavetable Synth 0')

In [62]:
for message in splice_track:
    if(message.is_meta):
        continue
    time.sleep(message.time * seconds_per_tick)
    port.send(message)
    